# Setup

In [1]:
%pip install ipywidgets
%pip install duckdb

Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.4/20.4 MB 65.6 MB/s eta 0:00:00 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
sys.path.append('../scripts')

# Extract the Features

In [3]:
import os
import features
from log import log


def extract_features(dataset: str = "stackoverflow_dba", version: str = "v1.0", sqlstorm: bool = True):
    os.makedirs(f"features/{version}", exist_ok=True)
    input_file = f"../results/{version}/features_raw/{dataset}{'_sqlstorm_' + version if sqlstorm else ''}.csv"
    output_file = f"features/{version}/{dataset}{'_sqlstorm' if sqlstorm else ''}.csv"

    log.info(f"Extracting features for {dataset} version {version} (SQLStorm: {sqlstorm})")
    with log.file(output_file.replace(".csv", ".log")) as log_file:
        features.compute(input_file, output_file=output_file)


extract_features(dataset="job", version="v0.0", sqlstorm=False)
extract_features(dataset="tpchSf1", version="v0.0", sqlstorm=False)
extract_features(dataset="tpcdsSf1", version="v0.0", sqlstorm=False)

extract_features(dataset="stackoverflow_dba", version="v1.0", sqlstorm=True)
extract_features(dataset="job", version="v1.0", sqlstorm=True)
extract_features(dataset="tpchSf1", version="v1.0", sqlstorm=True)
extract_features(dataset="tpcdsSf1", version="v1.0", sqlstorm=True)

[07:00:10]  INFO        Extracting features for job version v0.0 (SQLStorm:     
                        False)                                                  
            INFO        Extracting features for tpchSf1 version v0.0 (SQLStorm: 
                        False)                                                  
            INFO        Extracting features for tpcdsSf1 version v0.0           
                        (SQLStorm: False)                                       
            INFO        Extracting features for stackoverflow_dba version v1.0  
                        (SQLStorm: True)                                        
[07:00:54]  INFO        Extracting features for job version v1.0 (SQLStorm:     
                        True)                                                   
[07:01:19]  INFO        Extracting features for tpchSf1 version v1.0 (SQLStorm: 
                        True)                                                   
[07:01:55]  INFO        Extr

# Load the Features

In [4]:
import duckdb


def load_features(dataset: str = "stackoverflow_dba", version: str = "v1.0", sqlstorm: bool = True):
    file = f"features/{version}/{dataset}{'_sqlstorm' if sqlstorm else ''}.csv"
    features = duckdb.read_csv(file)
    operators = duckdb.read_csv(file.replace(".csv", "_operators.csv"))
    expressions = duckdb.read_csv(file.replace(".csv", "_expressions.csv"))

    operators2 = duckdb.sql("select query, operator from operators where operator <> 'Result' and operator <> 'GroupJoin' and operator <> 'Map' and operator <> 'IterationScan' and operator <> 'InlineTable'"
                            "union all select query, 'Join' as operator from operators where operator = 'GroupJoin'"
                            "union all select query, 'GroupBy' as operator from operators where operator = 'GroupJoin'")
    expressions2 = duckdb.sql("select query, expression, category, type from expressions where category <> 'base'")

    return features, operators2, expressions2


job_small, job_small_ops, job_small_exprs = load_features(dataset="job", version="v0.0", sqlstorm=False)
tpch_small, tpch_small_ops, tpch_small_exprs = load_features(dataset="tpchSf1", version="v0.0", sqlstorm=False)
tpcds_small, tpcds_small_ops, tpcds_small_exprs = load_features(dataset="tpcdsSf1", version="v0.0", sqlstorm=False)

so, so_ops, so_exprs = load_features(dataset="stackoverflow_dba", version="v1.0", sqlstorm=True)
job, job_ops, job_exprs = load_features(dataset="job", version="v1.0", sqlstorm=True)
tpch, tpch_ops, tpch_exprs = load_features(dataset="tpchSf1", version="v1.0", sqlstorm=True)
tpcds, tpcds_ops, tpcds_exprs = load_features(dataset="tpcdsSf1", version="v1.0", sqlstorm=True)

# Analysis

In [5]:

result = duckdb.sql("""
           select   complexity, 
                    count(*)::double num_queries,
                    avg(querylength) avg_querylength,
                    avg((select count(*) from so_ops o where o.query = f.query))  as avg_ops,
                    avg((select count(*) from so_ops e where e.query = f.query and operator = 'Join'))  as avg_joins,    
                    avg((select count(*) from so_exprs e where e.query = f.query))  as avg_exprs,           
           from so f
           group by complexity
           union all
           select  'all' complexity, 
                    count(*)::double num_queries,
                    avg(querylength) avg_querylength,
                    avg((select count(*) from so_ops o where o.query = f.query))  as avg_ops,
                    avg((select count(*) from so_ops e where e.query = f.query and operator = 'Join'))  as avg_joins,    
                    avg((select count(*) from so_exprs e where e.query = f.query))  as avg_exprs,
           from so f
           union all
           select 'job_small' complexity,
                    count(*)::double num_queries,
                    avg(querylength) avg_querylength,
                    avg((select count(*) from job_small_ops o where o.query = f.query))  as avg_ops,
                    avg((select count(*) from job_small_ops e where e.query = f.query and operator = 'Join'))  as avg_joins,    
                    avg((select count(*) from job_small_exprs e where e.query = f.query))  as avg_exprs,
           from job_small f
           union all
           select 'tpch_small' complexity,
                    count(*)::double num_queries,
                    avg(querylength) avg_querylength,
                    avg((select count(*) from tpch_small_ops o where o.query = f.query))  as avg_ops,
                    avg((select count(*) from tpch_small_ops e where e.query = f.query and operator = 'Join'))  as avg_joins,    
                    avg((select count(*) from tpch_small_exprs e where e.query = f.query))  as avg_exprs,
           from tpch_small f
           union all
           select 'tpcds_small' complexity,
                    count(*)::double num_queries,
                    avg(querylength) avg_querylength,
                    avg((select count(*) from tpcds_small_ops o where o.query = f.query))  as avg_ops,
                    avg((select count(*) from tpcds_small_ops e where e.query = f.query and operator = 'Join'))  as avg_joins,    
                    avg((select count(*) from tpcds_small_exprs e where e.query = f.query))  as avg_exprs,
            from tpcds_small f
            union all
            select 'job' complexity,
                    count(*)::double num_queries,
                    avg(querylength) avg_querylength,
                    avg((select count(*) from job_ops o where o.query = f.query))  as avg_ops,
                    avg((select count(*) from job_ops e where e.query = f.query and operator = 'Join'))  as avg_joins,    
                    avg((select count(*) from job_exprs e where e.query = f.query))  as avg_exprs,
            from job f
            union all
            select 'tpch' complexity,
                    count(*)::double num_queries,
                    avg(querylength) avg_querylength,
                    avg((select count(*) from tpch_ops o where o.query = f.query))  as avg_ops,
                    avg((select count(*) from tpch_ops e where e.query = f.query and operator = 'Join'))  as avg_joins,    
                    avg((select count(*) from tpch_exprs e where e.query = f.query))  as avg_exprs,
            from tpch f
            union all
            select 'tpcds' complexity,
                    count(*)::double num_queries,
                    avg(querylength) avg_querylength,
                    avg((select count(*) from tpcds_ops o where o.query = f.query))  as avg_ops,
                    avg((select count(*) from tpcds_ops e where e.query = f.query and operator = 'Join'))  as avg_joins,    
                    avg((select count(*) from tpcds_exprs e where e.query = f.query))  as avg_exprs
            from tpcds f
""").show()

┌─────────────┬─────────────┬────────────────────┬────────────────────┬────────────────────┬────────────────────┐
│ complexity  │ num_queries │  avg_querylength   │      avg_ops       │     avg_joins      │     avg_exprs      │
│   varchar   │   double    │       double       │       double       │       double       │       double       │
├─────────────┼─────────────┼────────────────────┼────────────────────┼────────────────────┼────────────────────┤
│ high        │      3460.0 │ 1347.1583815028903 │  22.07687861271676 │  6.199421965317919 │  35.61387283236994 │
│ low         │      4596.0 │  333.9264577893821 │  6.422758920800696 │ 1.8748912097476067 │  8.137946040034812 │
│ medium      │     10195.0 │ 1168.0664051005394 │ 14.407552721922512 │  4.166552231486023 │ 29.123589995095635 │
│ all         │     18251.0 │  991.9637828064216 │ 13.850747904224425 │ 3.9748506931127063 │ 25.069366062133582 │
│ job_small   │       113.0 │  825.0442477876106 │ 17.292035398230087 │  7.6460176991150

In [6]:
result = duckdb.sql("""
                    with ops as (select distinct operator from so_ops),
                         cops as (select complexity, operator, 
                                        count(*)::double / (select count(*) from so f2 where f2.complexity = f.complexity)::double as c 
                                    from so_ops o, so f where o.query = f.query group by complexity, operator),
                         cops_tpch as (select operator, 
                                        count(*)::double / (select count(*) from tpch_small)::double as c 
                                    from tpch_small_ops o group by operator),
                         cops_tpcds as (select operator, 
                                        count(*)::double / (select count(*) from tpcds_small)::double as c 
                                    from tpcds_small_ops o group by operator)
                    select ops.operator, 
                        (select c from cops c where c.operator = ops.operator and c.complexity = 'low') as low,
                        (select c from cops c where c.operator = ops.operator and c.complexity = 'medium') as medium,
                        (select c from cops c where c.operator = ops.operator and c.complexity = 'high') as high,
                        (select c from cops_tpch c where c.operator = ops.operator) as tpch,
                        (select c from cops_tpcds c where c.operator = ops.operator) as tpcds
                    from ops
                    where ops.operator not in ('PipelineBreakerScan', 'Temp')
                    order by low desc nulls last, medium desc nulls last, high desc nulls last
""")
result.show()

for r in result.fetchall():
    def conv(c):
        return f"0" if c is None else (f"{c:.3f}" if c < 0.01 else f"{c:.2f}")
    print(f"{r[0]} & {conv(r[1])} & {conv(r[2])} & {conv(r[3])} & {conv(r[4])} & {conv(r[5])} \\\\")

┌──────────────┬───────────────────────┬───────────────────────┬───────────────────────┬─────────────────────┬─────────────────────┐
│   operator   │          low          │        medium         │         high          │        tpch         │        tpcds        │
│   varchar    │        double         │        double         │        double         │       double        │       double        │
├──────────────┼───────────────────────┼───────────────────────┼───────────────────────┼─────────────────────┼─────────────────────┤
│ TableScan    │    2.8729329852045256 │     4.939676311917607 │     5.749132947976879 │  3.6818181818181817 │   7.223300970873787 │
│ Join         │    1.8748912097476067 │     4.166552231486023 │     6.199421965317919 │  2.8181818181818183 │   6.300970873786408 │
│ Sort         │    0.9873803307223673 │    1.0277587052476704 │    1.0965317919075144 │  0.8181818181818182 │  0.8349514563106796 │
│ GroupBy      │    0.6818973020017406 │    2.3321235899950956 │    4

In [7]:
result = duckdb.sql("""
                    with exps as (select distinct category from so_exprs),
                         cexps as (select complexity, category, count(*)::double / (select count(*) from so f2 where f2.complexity = f.complexity)::double as c from so_exprs o, so f where o.query = f.query group by complexity, category),
                            cexps_tpch as (select category, count(*)::double / (select count(*) from tpch_small)::double as c from tpch_small_exprs o group by category),
                            cexps_tpcds as (select category, count(*)::double / (select count(*) from tpcds_small)::double as c from tpcds_small_exprs o group by category)
                    select exps.category, 
                        (select c from cexps c where c.category = exps.category and c.complexity = 'low') as low,
                        (select c from cexps c where c.category = exps.category and c.complexity = 'medium') as medium,
                        (select c from cexps c where c.category = exps.category and c.complexity = 'high') as high,
                        (select c from cexps_tpch c where c.category = exps.category) as tpch,
                        (select c from cexps_tpcds c where c.category = exps.category) as tpcds
                    from exps
                    order by low desc nulls last, medium desc nulls last, high desc nulls last
""")
result.show()

for r in result.fetchall():
    def conv(c):
        return f"0" if c is None else (f"{c:.3f}" if c < 0.01 else f"{c:.2f}")
    print(f"{r[0]} & {conv(r[1])} & {conv(r[2])} & {conv(r[3])} & {conv(r[4])} & {conv(r[5])} \\\\")

┌─────────────────────┬──────────────────────┬──────────────────────┬────────────────────────┬─────────────────────┬──────────────────────┐
│      category       │         low          │        medium        │          high          │        tpch         │        tpcds         │
│       varchar       │        double        │        double        │         double         │       double        │        double        │
├─────────────────────┼──────────────────────┼──────────────────────┼────────────────────────┼─────────────────────┼──────────────────────┤
│ comparison_low      │    4.001087902523934 │   10.753310446297204 │     15.035260115606937 │   7.090909090909091 │    24.83495145631068 │
│ agg_low             │     2.93668407310705 │    8.926630701324179 │      9.128901734104046 │   2.409090909090909 │     4.41747572815534 │
│ cast                │   0.7071366405570061 │    2.758509073075037 │      2.538150289017341 │  0.7727272727272727 │   2.4563106796116503 │
│ case              